#Mount Gdrive

In [ ]:
#Mounting my Google Drive to store and load files
from google.colab import drive
drive.mount('/content/gdrive/')

#Create a symbolic link
!ln -s /content/gdrive/MyDrive/ /mydrive

Mounted at /content/gdrive/


#Clone and setup darknet repository

In [ ]:
# Clone darknet repo
%%capture
!git clone https://github.com/AlexeyAB/darknet

# Change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

# Make darknet (build)
!make

#Load dataset

In [ ]:
#loading the train and test datasets
%cd data
#Train data
!unzip -q '/mydrive/Data/obj.zip'
#Test data
!unzip -q '/mydrive/Data/test.zip'
%cd ..

/content/darknet/data
/content/darknet


In [ ]:
# Define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

#Custom training files

In [ ]:
# Create a copy of config file and edit it.
!cp cfg/yolov3.cfg cfg/yolov3_custom.cfg

#Copy customized files from gdrive
!cp /mydrive/yolov3/obj.names /content/darknet/data/
!cp /mydrive/yolov3/obj.data /content/darknet/data/
!cp /mydrive/yolov3/yolov3_custom.cfg /content/darknet/cfg/
!cp /mydrive/yolov3/generate_train.py /content/darknet/
!cp /mydrive/yolov3/generate_test.py /content/darknet/

In [ ]:
#Run scripts to generate the paths for training and test images
!python generate_train.py
!python generate_test.py

#YOLO Training

In [ ]:
# Download pretrained convolutional layer weights (Run only when trianing for the first time)
!wget http://pjreddie.com/media/files/darknet53.conv.74

# train your custom detector  (Run only when trianing for the first time)
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show

In [ ]:
#Incase training got interrupted resume from the last checkpoint instead of training from start
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg /mydrive/yolov3/backup/yolov3_custom_last.weights -dont_show

In [ ]:
#Show learning curve
imShow('chart.png')

#Model Evaluation

In [ ]:
#Calculate the Mean Average Precis
!./darknet detector map data/obj.data cfg/yolov3_custom.cfg /mydrive/yolov3/backup/yolov3_custom_last.weights

#Model Inference

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3_custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3_custom.cfg
%cd ..

/content/darknet/cfg
/content/darknet


##Image

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov3_custom.cfg /mydrive/yolov3/backup/yolov3_custom_last.weights /mydrive/Data/test1.jpg
imShow('predictions.jpg')

##Video

In [ ]:
#Test the algorithm on video file
!./darknet detector demo data/obj.data cfg/yolov3_custom.cfg /mydrive/yolov3/backup/yolov3_custom_last.weights -dont_show -ext_output /mydrive/Data/vid_3.mp4 -i 0 -out_filename custom_res.avi